In [ ]:
!pip install openai

# Simple Chat Completion

In [ ]:
from openai import OpenAI

markdown_text = """
# This is a heading

This is some **bold** text, and this is some *italic* text.

Here's a list:

* Item 1
* Item 2

"""

OAI_KEY="INSERT YOUR KEY HERE"
client = OpenAI(api_key=OAI_KEY)

completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system", "content": "You are a helpful assistant."
        },
        {
            "role": "user",
            "content": "Write a haiku about recursion in programming."
        }
    ]
)

print(completion.choices[0].message)


In [ ]:
# imports
import os
import time  # for measuring time duration of API calls
from openai import OpenAI

import json
from pprint import pprint
from textwrap import dedent
from IPython.display import Markdown, JSON, display, Math

display(Markdown(completion.choices[0].message.content))


# Chat Completions using Streaming

In [ ]:
# Initialize OAI client
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", OAI_KEY))

In [ ]:
# Example of an OpenAI ChatCompletion request
# https://platform.openai.com/docs/guides/text-generation/chat-completions-api

# record the time before the request is sent
start_time = time.time()

# send a ChatCompletion request to count to 100
response = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=[
        {'role': 'user', 'content': 'Count to 100, with a comma between each number and no newlines. E.g., 1, 2, 3, ...'}
    ],
    temperature=0,
)
# calculate the time it took to receive the response
response_time = time.time() - start_time

# print the time delay and text received
print(f"Full response received {response_time:.2f} seconds after request")
print(f"Full response received:\n{response}")


In [ ]:
reply = response.choices[0].message
display(Markdown(f"Extracted reply: \n{reply}"))

reply_content = response.choices[0].message.content
display(Markdown(f"Extracted content: \n{reply_content}"))


In [ ]:
# Example of an OpenAI ChatCompletion request with stream=True
# https://platform.openai.com/docs/api-reference/streaming#chat/create-stream

# a ChatCompletion request
response = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=[
        {'role': 'user', 'content': "What's 1+1? Answer in one word."}
    ],
    temperature=0,
    stream=True
)

# This prints the response as it streams back from the API call
for chunk in response:
    print(chunk.choices[0].delta.content)

### Get token usage data for streamed chat completion responses

In [ ]:
# Example of an OpenAI ChatCompletion request with stream=True and stream_options={"include_usage": True}

# a ChatCompletion request
response = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=[
        {'role': 'user', 'content': "What's 1+1? Answer in one word."}
    ],
    temperature=0,
    stream=True,
    stream_options={"include_usage": True}, # retrieving token usage for stream response
)

for chunk in response:
    print(f"choices: {chunk.choices}\nusage: {chunk.usage}")
    print("****************")

# Structured Outputs

In [ ]:
# init client
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", OAI_KEY))

MODEL = "gpt-4o-2024-08-06"

In [ ]:
math_tutor_prompt = '''
    You are a helpful math tutor. You will be provided with a math problem,
    and your goal will be to output a step by step solution, along with a final answer.
    For each step, just provide the output as an equation use the explanation field to detail the reasoning.
'''

def get_math_solution(question):
    response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {
            "role": "system", 
            "content": dedent(math_tutor_prompt)
        },
        {
            "role": "user", 
            "content": question
        }
    ],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "math_reasoning",
            "schema": {
                "type": "object",
                "properties": {
                    "steps": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "explanation": {"type": "string"},
                                "output": {"type": "string"}
                            },
                            "required": ["explanation", "output"],
                            "additionalProperties": False
                        }
                    },
                    "final_answer": {"type": "string"}
                },
                "required": ["steps", "final_answer"],
                "additionalProperties": False
            },
            "strict": True
        }
    }
    )

    return response.choices[0].message

In [ ]:
# Testing with an example question
question = "how can I solve 8x + 7 = -23"

result = get_math_solution(question)
display(Markdown(pprint(json.loads(result.content), depth=3, indent=4, width=160)))

In [ ]:
def print_math_response(response):
    result = json.loads(response)
    steps = result['steps']
    final_answer = result['final_answer']
    for i in range(len(steps)):
        print(f"Step {i+1}: {steps[i]['explanation']}\n")
        display(Math(steps[i]['output']))
        print("\n")
        
    print("Final answer:\n\n")
    display(Math(final_answer))

In [ ]:
print_math_response(result.content)

### Use `pydantic` to define a BaseModel and `parsed` helper

In [ ]:
from pydantic import BaseModel

class MathReasoning(BaseModel):
    class Step(BaseModel):
        explanation: str
        output: str

    steps: list[Step]
    final_answer: str

def get_math_solution(question: str):
    completion = client.beta.chat.completions.parse(
        model=MODEL,
        messages=[
            {"role": "system", "content": dedent(math_tutor_prompt)},
            {"role": "user", "content": question},
        ],
        response_format=MathReasoning,
    )

    return completion.choices[0].message

In [ ]:
result = get_math_solution(question).parsed

In [ ]:
# Uses model defined above
for step in result.steps:
    display(Markdown(step.explanation))
    display(Markdown(step.output))

display(Markdown(f"Final Answer: {result.final_answer}"))